In [1]:
import tensorflow as tf
import keras
import keras_nlp
print("Tensorflow 2.16.2 is expected. The running version is", tf.__version__)
print("Keras 3.4.1 is expected. The running version is", keras.__version__)
print("KerasNLP 0.12.1 is expected. The running version is", keras_nlp.__version__)

import numpy as np
import tensorflow_datasets as tfds
import datasets

Tensorflow 2.16.2 is expected. The running version is 2.16.2
Keras 3.4.1 is expected. The running version is 3.4.1
KerasNLP 0.12.1 is expected. The running version is 0.12.1


In [2]:
import nltk

def word_stats(text):
    tokenized_text = nltk.word_tokenize(text)
    print("Words:", len(tokenized_text), "Kinds of words:", len(set(tokenized_text)))

***
# CNN/Daily Mail

In [3]:
train_ds, validation_ds, test_ds = tfds.load(
    'huggingface:ccdv__cnn_dailymail/3.0.0',
    split=['train', 'validation', 'test'],
    builder_kwargs={
        'trust_remote_code': True,
    },
)

/Users/mitsuaki.ishimoto/.pyenv/versions/3.10.11/lib/python3.10/site-packages/tensorflow_datasets/core/dataset_builders/huggingface_dataset_builder.py:160: FutureWarning: list_datasets is deprecated and will be removed in the next major version of datasets. Use 'huggingface_hub.list_datasets' instead.
  hf_names = hf_datasets.list_datasets()
2024-07-15 00:28:09.684392: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2 Pro
2024-07-15 00:28:09.684421: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 32.00 GB
2024-07-15 00:28:09.684425: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 10.67 GB
2024-07-15 00:28:09.684446: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-07-15 00:28:09.684461: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Creat

In [4]:
df = tfds.as_dataframe(train_ds)
print(len(df))
df.head(5)

2024-07-15 00:28:26.880399: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


287113


,article,highlights,id
0,b'It\'s official: U.S. President Barack Obama ...,b'Syrian official: Obama climbed to the top of...,b'0001d1afc246a7964130f43ae940af6bc6c57f01'
1,b'(CNN) -- Usain Bolt rounded off the world ch...,"b""Usain Bolt wins third gold of world champion...",b'0002095e55fcbd3a2f366d9bf92a95433dc305ef'
2,"b'Kansas City, Missouri (CNN) -- The General S...",b'The employee in agency\'s Kansas City office...,b'00027e965c8264c35cc1bc55556db388da82b07f'
3,b'Los Angeles (CNN) -- A medical doctor in Van...,b'NEW: A Canadian doctor says she was part of ...,b'0002c17436637c4fe1837c935c04de47adb18e9a'
4,b'(CNN) -- Police arrested another teen Thursd...,b'Another arrest made in gang rape outside Cal...,b'0003ad6ef0c37534f80b55b4235108024b407f0b'


In [5]:
"""
The first code takes seconds to simply count words.
On the other hand, the latter code that can precisely tokenize takes more than 7 minutes.
"""
df['highlights_count'] = df['highlights'].apply(lambda x: len(x.split()))
df['article_count'] = df['article'].apply(lambda x: len(x.split()))

# import nltk

# def count_words(s):
#     s = s.decode('utf-8')
#     return len(nltk.word_tokenize(s))

# df['highlights_count'] = df['highlights'].map(count_words)
# df['article_count'] = df['article'].map(count_words)
df.describe()

,highlights_count,article_count
count,287113.000000,287113.000000
mean,51.478118,691.626074
std,21.186010,336.493356
min,4.000000,8.000000
25%,38.000000,443.000000
50%,48.000000,632.000000
75%,60.000000,877.000000
max,1296.000000,2347.000000


In [6]:
def decode(s):
    return s.decode('utf-8')

df['highlights_str'] = df['highlights'].map(decode)
highlights = df['highlights_str'].str.cat(sep=' ')

df['article_str'] = df['article'].map(decode)
article = df['article_str'].str.cat(sep=' ')

In [7]:
word_stats(highlights)
word_stats(article)

Words: 15699963 Kinds of words: 261325
Words: 226308079 Kinds of words: 936405


(None, None)

***
# BillSum

In [8]:
ds = datasets.load_dataset(
    "FiscalNote/billsum",
    trust_remote_code=False
)

In [9]:
ds.set_format("pandas")
df = ds['train'][:]
print(len(df))
df.head(5)

18949


,text,summary,title
0,SECTION 1. LIABILITY OF BUSINESS ENTITIES PROV...,Shields a business entity from civil liability...,A bill to limit the civil liability of busines...
1,SECTION 1. SHORT TITLE.\n\n This Act may be...,Human Rights Information Act - Requires certai...,Human Rights Information Act
2,SECTION 1. SHORT TITLE.\n\n This Act may be...,Jackie Robinson Commemorative Coin Act - Direc...,Jackie Robinson Commemorative Coin Act
3,SECTION 1. NONRECOGNITION OF GAIN WHERE ROLLOV...,Amends the Internal Revenue Code to provide (t...,To amend the Internal Revenue Code to provide ...
4,SECTION 1. SHORT TITLE.\n\n This Act may be...,Native American Energy Act - (Sec. 3) Amends t...,Native American Energy Act


In [10]:
df['title_count'] = df['title'].apply(lambda x: len(x.split()))
df['summary_count'] = df['summary'].apply(lambda x: len(x.split()))
df['text_count'] = df['text'].apply(lambda x: len(x.split()))
df.describe()

,title_count,summary_count,text_count
count,18949.000000,18949.000000,18949.000000
mean,18.461977,179.119268,1289.393055
std,14.413841,115.694566,518.105838
min,1.000000,8.000000,194.000000
25%,7.000000,92.000000,857.000000
50%,14.000000,157.000000,1166.000000
75%,27.000000,240.000000,1644.000000
max,137.000000,808.000000,3055.000000


In [11]:
summary = df['summary'].str.cat(sep=' ')
title = df['title'].str.cat(sep=' ')
text = df['text'].str.cat(sep=' ')
word_stats(summary)
word_stats(title)
word_stats(text)

Words: 3868277 Kinds of words: 41516
Words: 375887 Kinds of words: 15213
Words: 31573067 Kinds of words: 131637


(None, None, None)

***
# aeslc

In [12]:
ds = datasets.load_dataset(
    "Yale-LILY/aeslc",
    trust_remote_code=False
)

In [13]:
ds.set_format("pandas")
df = ds['train'][:]
print(len(df))
df.head(5)

14436


,email_body,subject_line
0,"Greg/Phillip, Attached is the Grande Communic...",Service Agreement
1,Phillip & Keith Attached is the first draw re...,Bishops Corner
2,Your Internet Banking accounts are now setup a...,Internet Banking
3,To our IBS Customers that are still hanging in...,Internet Banking
4,Phillip Good Morning!\nI hope you had a wonder...,SMEs for expert stories


In [14]:
df['email_body_count'] = df['email_body'].apply(lambda x: len(x.split()))
df['subject_line_count'] = df['subject_line'].apply(lambda x: len(x.split()))
df.describe()

,email_body_count,subject_line_count
count,14436.000000,14436.000000
mean,118.231366,3.975686
std,148.968600,2.553487
min,25.000000,1.000000
25%,45.000000,2.000000
50%,74.000000,3.000000
75%,131.000000,5.000000
max,3136.000000,15.000000


In [15]:
email_body = df['email_body'].str.cat(sep=' ')
subject_line = df['subject_line'].str.cat(sep=' ')
word_stats(email_body)
word_stats(subject_line)

Words: 1968415 Kinds of words: 69744
Words: 62964 Kinds of words: 12018


(None, None)

***
# booksum

In [16]:
ds = datasets.load_dataset(
    "kmfoda/booksum",
    trust_remote_code=False
)
ds.set_format("pandas")
df = ds['train'][:]
print(len(df))
df.head(5)

9600


,bid,is_aggregate,source,chapter_path,summary_path,book_id,summary_id,content,summary,chapter,chapter_length,summary_name,summary_url,summary_text,summary_analysis,summary_length,analysis_length
0,27681,True,cliffnotes,all_chapterized_books/27681-chapters/chapters_...,finished_summaries/cliffnotes/The Last of the ...,The Last of the Mohicans.chapters 1-2,chapters 1-2,NaN,"{""name"": ""Chapters 1-2"", ""url"": ""https://web.a...","\n ""Mine ear is open, and my heart prepared:\...",6471.0,Chapters 1-2,https://web.archive.org/web/20201101053205/htt...,"Before any characters appear, the time and geo...",These two chapters introduce the reader to the...,388.0,473.0
1,27681,False,cliffnotes,all_chapterized_books/27681-chapters/03.txt,finished_summaries/cliffnotes/The Last of the ...,The Last of the Mohicans.chapter 3,chapter 3,NaN,"{""name"": ""Chapter 3"", ""url"": ""https://web.arch...","\n ""Before these fields were shorn and tilled...",3132.0,Chapter 3,https://web.archive.org/web/20201101053205/htt...,In another part of the forest by the river a f...,This chapter introduces the other three main a...,198.0,149.0
2,27681,False,cliffnotes,all_chapterized_books/27681-chapters/04.txt,finished_summaries/cliffnotes/The Last of the ...,The Last of the Mohicans.chapter 4,chapter 4,NaN,"{""name"": ""Chapter 4"", ""url"": ""https://web.arch...","\n ""Well, go thy way: thou shalt not from thi...",3075.0,Chapter 4,https://web.archive.org/web/20201101053205/htt...,When the mounted party from Fort Howard approa...,Since this chapter is mostly one of surface ac...,319.0,75.0
3,27681,False,cliffnotes,all_chapterized_books/27681-chapters/05.txt,finished_summaries/cliffnotes/The Last of the ...,The Last of the Mohicans.chapter 5,chapter 5,NaN,"{""name"": ""Chapter 5"", ""url"": ""https://web.arch...","\n ""In such a night\n Di...",3268.0,Chapter 5,https://web.archive.org/web/20201101053205/htt...,"The pursuit of Magua is unsuccessful, but Hawk...",Here the reader encounters the first bloodshed...,329.0,156.0
4,27681,False,cliffnotes,all_chapterized_books/27681-chapters/06.txt,finished_summaries/cliffnotes/The Last of the ...,The Last of the Mohicans.chapter 6,chapter 6,NaN,"{""name"": ""Chapter 6"", ""url"": ""https://web.arch...","\n ""Those strains that once did sweet in Zion...",3873.0,Chapter 6,https://web.archive.org/web/20201101053205/htt...,Heyward and the girls are uneasy and Gamut is ...,This chapter shows Cooper in his most inventiv...,321.0,128.0


In [17]:
df['summary_analysis_count'] = df['summary_analysis'].apply(lambda x: 0 if x is None else len(x.split()))
df['summary_text_count'] = df['summary_text'].apply(lambda x: 0 if x is None else len(x.split()))
df.describe()

,bid,content,chapter_length,summary_length,analysis_length,summary_analysis_count,summary_text_count
count,9600.000000,0.0,9600.000000,9600.000000,9600.000000,9600.000000,9600.00000
mean,5314.669375,NaN,3897.230625,376.896354,274.324063,272.387187,376.80375
std,10504.196539,NaN,4203.548176,331.915025,385.446081,383.108941,331.87908
min,11.000000,NaN,42.000000,2.000000,1.000000,0.000000,2.00000
25%,345.000000,NaN,1674.000000,171.000000,1.000000,0.000000,171.00000
50%,1254.000000,NaN,2779.000000,283.000000,133.000000,132.000000,283.00000
75%,2641.000000,NaN,4571.000000,467.000000,466.000000,465.000000,467.00000
max,45631.000000,NaN,114226.000000,4852.000000,5761.000000,5761.000000,4852.00000


***
# newsroom

In [ ]:
ds = datasets.load_dataset(
    "newsroom",
    trust_remote_code=True
)
ds.set_format("pandas")
df = ds['train'][:]
df.head(5)

***
# reddit

**Downloading and preparing dataset 2.93 GiB (download: 2.93 GiB, generated: 18.09 GiB, total: 21.01 GiB) to**

In [ ]:
ds = tfds.load('reddit')

***
# reddit_tifu

In [18]:
ds = datasets.load_dataset(
    "ctr4si/reddit_tifu", 'short',
    trust_remote_code=True
)
ds.set_format("pandas")
df = ds['train'][:]
print(len(df))
df.head(5)

79740


,ups,num_comments,upvote_ratio,score,documents,tldr,title
0,50.0,13.0,0.77,50.0,i was on skype on my tablet as i went to the t...,,forgetting to pull my underwear down before i ...
1,115.0,23.0,0.88,115.0,this actually happened a couple of years ago. ...,confuse a 5th grade girl for a boy in front of...,gender-stereotyping
2,14.0,11.0,0.78,14.0,i tend to leave half-drank beers in my garage....,,drinking a beer
3,16.0,12.0,0.79,16.0,"it was last october, but i'm feeling the fall-...","i found my estranged dad, thought i loved him ...",telling my dad that i love him.
4,0.0,2.0,0.42,0.0,"flashback to the past, almost exactly one year...",,how hard can you fail at something?


In [19]:
df['title_count'] = df['title'].apply(lambda x: 0 if x is None else len(x.split()))
df['documents_count'] = df['documents'].apply(lambda x: 0 if x is None else len(x.split()))
df['tldr_count'] = df['tldr'].apply(lambda x: 0 if x is None else len(x.split()))
df.describe()

,ups,num_comments,upvote_ratio,score,title_count,documents_count,tldr_count
count,79740.000000,79740.000000,79740.000000,79740.000000,79740.000000,79740.000000,79740.000000
mean,254.228363,37.300388,0.763873,254.228363,6.781703,308.035892,10.884098
std,1677.370361,155.337738,0.178532,1677.370361,3.709242,259.630296,14.005748
min,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000
25%,2.000000,3.000000,0.670000,2.000000,4.000000,148.000000,0.000000
50%,9.000000,7.000000,0.810000,9.000000,6.000000,242.000000,6.000000
75%,40.000000,18.000000,0.900000,40.000000,8.000000,387.000000,19.000000
max,96587.000000,4506.000000,1.000000,96587.000000,58.000000,6019.000000,457.000000


In [20]:
title = df['title'].str.cat(sep=' ')
documents = df['documents'].str.cat(sep=' ')
word_stats(title)
word_stats(documents)

Words: 592706 Kinds of words: 24227
Words: 28296784 Kinds of words: 169909


(None, None)

***
# samsum

In [ ]:
%pip install py7zr

In [21]:
ds = datasets.load_dataset(
    'Samsung/samsum',
    trust_remote_code=True
)
ds.set_format("pandas")
df = ds['train'][:]
print(len(df))
df.head(5)

14732


,id,dialogue,summary
0,13818513,Amanda: I baked cookies. Do you want some?\r\...,Amanda baked cookies and will bring Jerry some...
1,13728867,Olivia: Who are you voting for in this electio...,Olivia and Olivier are voting for liberals in ...
2,13681000,"Tim: Hi, what's up?\r\nKim: Bad mood tbh, I wa...",Kim may try the pomodoro technique recommended...
3,13730747,"Edward: Rachel, I think I'm in ove with Bella....",Edward thinks he is in love with Bella. Rachel...
4,13728094,Sam: hey overheard rick say something\r\nSam:...,"Sam is confused, because he overheard Rick com..."


In [22]:
df['summary_count'] = df['summary'].apply(lambda x: 0 if x is None else len(x.split()))
df['dialogue_count'] = df['dialogue'].apply(lambda x: 0 if x is None else len(x.split()))
df.describe()

,summary_count,dialogue_count
count,14732.000000,14732.000000
mean,20.317472,93.786383
std,11.153815,74.033457
min,1.000000,0.000000
25%,12.000000,39.000000
50%,18.000000,73.000000
75%,27.000000,128.000000
max,64.000000,803.000000


In [23]:
dialogue = df['dialogue'].str.cat(sep=' ')
summary = df['summary'].str.cat(sep=' ')
word_stats(dialogue)
word_stats(summary)

Words: 1827903 Kinds of words: 38786
Words: 345321 Kinds of words: 18160


(None, None)

***
# scientific_papers

In [ ]:
ds = tfds.load('scientific_papers')

**Downloading and preparing dataset 4.20 GiB (download: 4.20 GiB, generated: 7.07 GiB, total: 11.27 GiB) to**